In [14]:
# Dependencies
from bs4 import BeautifulSoup, SoupStrainer
import requests
import pandas as pd
from pprint import pprint
import re
import time

In [15]:
house_dict = []

for page in range(16, 21):
    linked = 'https://www.zillow.com/ca/condos/' + str(page) + '_p/?searchQueryState={"pagination":{"currentPage":' + str(page) + '},"usersSearchTerm":"ca","mapBounds":{"west":-153.847663625,"east":-84.765632375,"south":12.777434491046959,"north":56.013075839104054},"mapZoom":4,"regionSelection":[{"regionId":9,"regionType":2}],"filterState":{"sortSelection":{"value":"globalrelevanceex"},"isMultiFamily":{"value":false},"isManufactured":{"value":false},"isLotLand":{"value":false},"isTownhouse":{"value":false},"isApartment":{"value":false},"isComingSoon":{"value":false},"isAuction":{"value":false},"doz":{"value":"90"},"isSingleFamily":{"value":false}},"isListVisible":true,"isMapVisible":true}'
    user_agent = 'Mozilla/5.0 (Windows NT 5.1; rv:7.0.1) Gecko/20100101 Firefox/7.0.1'
    headers = {'User-Agent': user_agent}
    response = requests.get(linked, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    results = soup.find_all('article', class_='list-card')
    
    # Loop through returned results
    for result in results:
        # Error handling
        try:
            # Identify and return housing weblink
            house_website = str(result.a.get('href'))
            print(house_website)
            # Identify and return house data
            listing_type = result.find('div', class_="list-card-footer").text;
            price = result.find('div', class_="list-card-price").text
#             print(price)
            full_address = result.find('address', class_="list-card-addr").text
            street = result.find('address', class_="list-card-addr").text.split(",")[0]
            try:
                city = result.find('address', class_="list-card-addr").text.split(",")[1].replace(" ", "")
            except IndexError:
                city = 'null'
            try:
                state = result.find('address', class_="list-card-addr").text.split(",")[2].split(" ")[1]
            except IndexError:
                state = 'null'
            try:
                zip_code = result.find('address', class_="list-card-addr").text.split(",")[2].split(" ")[2]
            except IndexError:
                zip_code = 'null'
            bed = result.find('ul', class_="list-card-details").find('li').text.split(" ")[0]
            bath = result.find('ul', class_="list-card-details").select('li')[1].text.split(" ")[0]
            house_square_feet = result.find('ul', class_="list-card-details").select('li')[2].text.split(" ")[0]
            # Loop inside each house website and get more house data
            loop_response = requests.get(house_website, headers=headers)
            loop_soup = BeautifulSoup(loop_response.text, 'html.parser')
            loop_result1 = loop_soup.find('ul', class_="ds-home-fact-list")
            loop_result2 = loop_soup.findAll('li', class_="ds-home-fact-list-item")

            lot = 'null';
            house_type = 'null';
            year_build = 'null';
            parking = 'null';
            hoa = 'null';
            lot_conversion_to_sqft = 'null';
            zillow_price_per_sqft = 'null';
            
            for content in loop_result2:
                if "Lot" in content.text.split(":")[0]:
                    lot = content.text.split(":")[1];
                if "Type" in content.text.split(":")[0]:
                    house_type = content.text.split(":")[1];
                if "Year built" in content.text.split(":")[0]:
                    year_build = content.text.split(":")[1];
                if "Parking" in content.text.split(":")[0]:
                    parking = content.text.split(":")[1];
                if "HOA" in content.text.split(":")[0]:
                    hoa = content.text.split(":")[1];
                if "Price/sqft" in content.text.split(":")[0]:
                    zillow_price_per_sqft = content.text.split(":")[1];
                    
            try:
                if (str(lot.split(" ")[1]) == "sqft"):
                    lot_conversion_to_sqft = float(lot.split(" ")[0].replace(',', ""));
                elif ((str(lot.split(" ")[1]) == "acres") or (str(lot.split(" ")[1]) == "acre") or (str(lot.split(" ")[1]) == "Acres") or (str(lot.split(" ")[1]) == "Acre")):
                    lot_conversion_to_sqft = round(float(lot.split(" ")[0].replace(',', "")) * 43560, 2);
                elif (lot == "No Data"):
                    lot_conversion_to_sqft = 'null'
                else:
                    lot_conversion_to_sqft = 'null'
            except IndexError:
                lot_conversion_to_sqft = 'null'
                
            try:
                zestimate = loop_soup.findAll('span', class_="Text-c11n-8-33-0__aiai24-0 LoSpP")[0].text
            except:
                zestimate = 'null'
            try:
                rent_zestimate = loop_soup.findAll('span', class_="Text-c11n-8-33-0__aiai24-0 LoSpP")[1].text
                
            except:
                rent_zestimate = 'null'
            if (zestimate[0] != '$'):
                zestimate = 'null';
            print('zestimate', zestimate)
            print('rent zestimate', rent_zestimate)

            # Store data to house object if all elements are available
#             if (house_website and listing_type and price and full_address and street and city and state and zip_code and bed and bath and house_square_feet and house_type and year_build):
            house_obj = {
                "Listing_type": listing_type,
                "Price": price,
                "Full_address": full_address,
                "Street": street,
                "City": city,
                "State": state,
                "Zip_code": zip_code,
                "Bed": bed,
                "Bath": bath,
                "Square_Feet": house_square_feet,
                "Lot": lot,
                "Lot_conversion_to_sqft": lot_conversion_to_sqft,
                "House_type": house_type,
                "Year_build": year_build,
                "Parking": parking,
                "HOA": hoa,
                "Zestimate": zestimate,
                "Rent_zestimate": rent_zestimate,
                "Zillow_price/sqft": zillow_price_per_sqft,
                "House_website": house_website
            }

            #Continue to add data into collection house_dict
            house_dict.append(house_obj)
            time.sleep(3)
            
        except AttributeError as e:
            continue

print(house_dict)

https://www.zillow.com/homedetails/2013-E-Bard-Rd-Oxnard-CA-93033/16408944_zpid/
zestimate $439,596
rent zestimate $2,700/mo
https://www.zillow.com/homedetails/8276-Chappellet-Pl-Rancho-Cucamonga-CA-91730/17295706_zpid/
zestimate $412,350
rent zestimate $2,050/mo
https://www.zillow.com/homedetails/45452-Sunset-Ln-7-Palm-Desert-CA-92260/17749105_zpid/
zestimate $210,002
rent zestimate $2,400/mo
https://www.zillow.com/homedetails/12436-Bay-Hill-Ct-67-Garden-Grove-CA-92843/2071586568_zpid/
zestimate $421,944
rent zestimate null
https://www.zillow.com/homedetails/846-Cotati-Ter-UNIT-1-Sunnyvale-CA-94085/325805662_zpid/
zestimate $1,671,782
rent zestimate null
https://www.zillow.com/homedetails/8801-Willis-Ave-59-Panorama-City-CA-91402/20135084_zpid/
zestimate $290,002
rent zestimate $2,100/mo
https://www.zillow.com/homedetails/100-Atlantic-Ave-APT-402-Long-Beach-CA-90802/111373019_zpid/
zestimate $337,392
rent zestimate $1,700/mo
https://www.zillow.com/homedetails/46-Stone-St-UNIT-16-Salin

zestimate $1,004,910
rent zestimate $3,500/mo
https://www.zillow.com/homedetails/709-San-Conrado-Ter-UNIT-6-Sunnyvale-CA-94085/2072363168_zpid/
zestimate null
rent zestimate null
https://www.zillow.com/homedetails/13920-Moorpark-St-APT-206-Los-Angeles-CA-91423/19986759_zpid/
zestimate $777,001
rent zestimate $2,995/mo
https://www.zillow.com/homedetails/80-Fairymoss-Irvine-CA-92620/124718944_zpid/
zestimate $685,997
rent zestimate $2,850/mo
https://www.zillow.com/homedetails/507-Wickson-Ave-APT-106-Oakland-CA-94610/2071533486_zpid/
zestimate null
rent zestimate null
https://www.zillow.com/homedetails/1004-W-Balboa-Blvd-B-Newport-Beach-CA-92661/119684705_zpid/
zestimate $2,503,131
rent zestimate $8,472/mo
https://www.zillow.com/homedetails/9760-Mesa-Springs-Way-UNIT-39-San-Diego-CA-92126/16833769_zpid/
zestimate $2,200/mo
rent zestimate null
https://www.zillow.com/homedetails/15716-S-Normandie-Ave-APT-5-Gardena-CA-90247/20983352_zpid/
zestimate $480,438
rent zestimate $2,599/mo
https://w

https://www.zillow.com/homedetails/7304-Richfield-St-UNIT-103-Paramount-CA-90723/21044308_zpid/
zestimate $363,835
rent zestimate $2,150/mo
https://www.zillow.com/homedetails/35-Dolores-St-APT-310-San-Francisco-CA-94103/2071527393_zpid/
zestimate null
rent zestimate null
https://www.zillow.com/homedetails/50-Lansing-St-UNIT-413-San-Francisco-CA-94105/71670261_zpid/
zestimate $337,742
rent zestimate $2,855/mo
https://www.zillow.com/homedetails/7634-Hollister-Ave-UNIT-125-Goleta-CA-93117/15917571_zpid/
zestimate $612,823
rent zestimate $2,800/mo
https://www.zillow.com/homedetails/700-Promontory-Point-Ln-APT-1307-Foster-City-CA-94404/15633820_zpid/
zestimate $1,315,947
rent zestimate $5,000/mo
https://www.zillow.com/homedetails/6-Buckelew-St-Sausalito-CA-94965/2113276349_zpid/
zestimate $439,001
rent zestimate $2,695/mo
https://www.zillow.com/homedetails/1750-Radiance-Cmn-101-F9-Fremont-CA-94539/2071513835_zpid/
zestimate null
rent zestimate null
https://www.zillow.com/homedetails/784-Via

In [16]:
house_data = pd.DataFrame(house_dict)
house_data

Listing_type       Price  \
0                 REALTY WORLD GOLDEN ERA    $435,000   
1          REALTY MASTERS & ASSOCIATES, I    $400,000   
2           JACOB, CHRISTOPHER & LEE RLTY    $210,000   
3             THE ASSOCIATES REALTY GROUP    $412,000   
4                                          $1,599,888   
..                                    ...         ...   
155              EILEEN MCCARTHY R.E. INC    $579,000   
156                  PALISADE REALTY, INC    $410,000   
157          LEISURE LIVING RESALES, INC.    $325,000   
158  KELLER WILLIAMS REALTY CENTRAL COAST    $439,900   
159              TRG REALTY COMPANY, INC.    $439,000   

                                          Full_address  \
0                     2013 E Bard Rd, Oxnard, CA 93033   
1       8276 Chappellet Pl, Rancho Cucamonga, CA 91730   
2            45452 Sunset Ln #7, Palm Desert, CA 92260   
3        12436 Bay Hill Ct #67, Garden Grove, CA 92843   
4           846 Cotati Ter UNIT 1, Sunnyvale, CA 94085   
..                                                 ...   
155  4337 Marina City Dr UNIT 839, Marina Del Rey, ...   
156      3958 Louisiana St UNIT 6, San Diego, CA 92104   
157  13421 Del Monte Dr #24A-M14, Seal Beach, CA 90740   
158            1069 James Way, Arroyo Grande, CA 93420   
159        2135 Bellevue Ave #8, Los Angeles, CA 90026   

                           Street             City State Zip_code Bed Bath  \
0                  2013 E Bard Rd           Oxnard    CA    93033   3    3   
1              8276 Chappellet Pl  RanchoCucamonga    CA    91730   2    3   
2              45452 Sunset Ln #7       PalmDesert    CA    92260   1    2   
3           12436 Bay Hill Ct #67      GardenGrove    CA    92843   2    2   
4           846 Cotati Ter UNIT 1        Sunnyvale    CA    94085   4    4   
..                            ...              ...   ...      ...  ..  ...   
155  4337 Marina City Dr UNIT 839     MarinaDelRey    CA    90292   2    2   
156      3958 Louisiana St UNIT 6         SanDiego    CA    92104   2    2   
157   13421 Del Monte Dr #24A-M14        SealBeach    CA    90740   2    1   
158                1069 James Way     ArroyoGrande    CA    93420   2    2   
159          2135 Bellevue Ave #8       LosAngeles    CA    90026   1    1   

    Square_Feet   Lot Lot_conversion_to_sqft   House_type Year_build  \
0         1,289  null                   null        Condo       1980   
1           984  null                   null        Condo       1989   
2           996  null                   null        Condo       1961   
3           900  null                   null        Condo       1989   
4         1,903  null                   null        Condo       2019   
..          ...   ...                    ...          ...        ...   
155          --  null                   null        Condo    No Data   
156         782  null                   null        Condo       1982   
157         950  null                   null  Cooperative       1963   
158       1,320  null                   null        Condo       1980   
159         640  null                   null        Condo       1959   

                                               Parking             HOA  \
0                Garage, Garage Door Opener, On Street    $300 monthly   
1                                    Garage - Attached    $275 monthly   
2                                     Attached Carport    $290 monthly   
3    Garage Door Opener, Gated, Parking Space, Perm...    $395 monthly   
4                                    Garage - Attached    $304 monthly   
..                                                 ...             ...   
155                                           Assigned  $1,020 monthly   
156                                             Garage    $350 monthly   
157                           Assigned, Carport, Gated    $388 monthly   
158                                  Garage Faces Rear    $180 monthly   
159                            

In [17]:
# Export file as a CSV, without the Pandas index, but with the header
house_data.to_csv("Output/Zillow/Q1-2021_Condo_Data_page16-20.csv", index=False, header=True)